## Análise de Vendas do Dataset Olist

### Introdução
Neste notebook, vamos explorar e analisar o dataset de vendas da Olist para responder a perguntas específicas sobre o comportamento de vendas e padrões sazonais.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import folium
from folium.plugins import HeatMap
import geopandas as gpd
import os

sns.set_theme(style="whitegrid")

data_dir = '../dados/'

In [ ]:
customers = pd.read_csv (data_dir + 'olist_customers_dataset.csv')
geolocation = pd.read_csv (data_dir + 'olist_geolocation_dataset.csv')
order_items = pd.read_csv (data_dir + 'olist_order_items_dataset.csv')
order_payment = pd.read_csv (data_dir + 'olist_order_payments_dataset.csv')
order_reviews = pd.read_csv (data_dir + 'olist_order_reviews_dataset.csv')
orders = pd.read_csv (data_dir + 'olist_orders_dataset.csv')
products = pd.read_csv (data_dir + 'olist_products_dataset.csv')
sellers = pd.read_csv (data_dir + 'olist_sellers_dataset.csv')
product_category = pd.read_csv (data_dir + 'product_category_name_translation.csv')

Após importar as bibliotecas e os dados necessários, é importante realizar uma análise inicial dos dados. Verifiquei as dimensões dos datasets, colunas e tipos de dados.

In [ ]:
customers.head(), geolocation.head(), order_items.head(), order_payment.head(), order_reviews.head(), 
orders.head(), products.head(), sellers.head(), product_category.head()

In [ ]:
print(customers.info())
print(orders.info())
print(product_category.info())
print(orders.columns)
print(customers.columns)

As primeiras perguntas a serem respondidas foram: Qual é a distribuição do número de pedidos e qual é o valor médio dos pedidos, por estado?

Para sanar essas dúvidas, é necessário agrupar os pedidos por estado e calcular a soma e a média de cada estado.

In [ ]:
orders_by_state = customers.groupby('customer_state')['customer_id'].count().reset_index()
orders_by_state.columns = ['Estado', 'Quantidade de pedidos']

plt.figure(figsize=(14, 8))
sns.barplot(x='Estado', y='Quantidade de pedidos', data=orders_by_state)
plt.title('Números de pedidos por estado')
plt.xticks(rotation=45)
plt.show()

In [ ]:
orders_with_state = pd.merge(orders, customers[['customer_id', 'customer_state']], on='customer_id', how='left')

avg_orders_by_state = orders_with_state.groupby('customer_state')['order_id'].count().reset_index()
avg_orders_by_state.columns = ['Estado', 'Média de pedidos']

media_de_pedidos_por_estado = avg_orders_by_state['Média de pedidos'].mean()
print(f'Média de Pedidos por Estado: {media_de_pedidos_por_estado: .2f}')

Irei criar um mapa de calor (HeatMap), com a ajuda da biblioteca Folium.
Esse mapa me ajudará a entender melhor qual é a distribuição dos clientes por região, que é uma das perguntas a serem respondidas.

In [ ]:
map = folium.Map(location=[-14.235004, -51.92528], zoom_start= 4)
heat_data = [[row['geolocation_lat'], row['geolocation_lng']]for index, row in geolocation.iterrows()]
HeatMap(heat_data).add_to(map)
map.save('heat_map.html')

map

Para a próxima análise, vou focar em categoria de cada produto. Iremos descobrir categórias mais vendidas e qual categória gera a maior receita.

Como as vendas se distribuem entre diferentes categorias de produtos? Quais categorias têm melhor desempenho em termos de receita e volume?
Para responder essas questões, foi necessário fazer a junção de dois DataSets e agrupar a quantidade de pedidos em cada categoria. Após isso, ordenei o resultado do volume de pedidos do maior para o menor.
O mesmo foi feito para descobrir qual categoria gera mais receita, mas ao contrário do volume que contamos, a receita nos somamos o valor.

In [ ]:
orders_with_category = pd.merge(order_items, products[['product_id', 'product_category_name']], on='product_id', how='left')

count_by_category = orders_with_category.groupby('product_category_name')['product_id'].count().reset_index()
count_by_category.columns = ['Categoria', 'Quantidade de pedidos']
count_by_category_sorted = count_by_category.sort_values(by='Quantidade de pedidos', ascending=False).reset_index(drop=True)

sum_by_category = orders_with_category.groupby('product_category_name')['price'].sum().reset_index()
sum_by_category.columns = ['Categoria', 'Receita Gerada']
sum_by_category_sorted = sum_by_category.sort_values(by='Receita Gerada', ascending=False).reset_index(drop=True)

print(sum_by_category_sorted.head(10))
print(count_by_category_sorted.head(10))

Qual é o valor médio das compras (valor médio do pedido)?

In [ ]:
price_by_order = order_payment.groupby('order_id')['payment_value'].sum().reset_index()
price_by_order.columns = ['order id', 'price']
avg_price_by_order = price_by_order['price'].mean()

print(f'O valor médio do pedido é: {avg_price_by_order: .1f}')

Ao se questionar qual o volume de vendas para cada mês, representamos os dados em um gráfico de barras para uma melhor vizualização.

In [ ]:
orders['order_purchase_timestamp'] = pd.to_datetime(orders['order_purchase_timestamp'])
orders['year_month'] = orders['order_purchase_timestamp'].dt.to_period('M')

monthly_sales = orders.groupby('year_month')['order_id'].count().reset_index()
monthly_sales.columns = ['Quantidade de pedidos', 'Mês']

plt.figure(figsize=(14, 8))
sns.barplot(y='Mês', x='Quantidade de pedidos', data=monthly_sales)
plt.title('Números de pedidos por mês')
plt.xticks(rotation=45)
plt.show()